# Exogenous Variables and Time Series Forecasting

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/gautamnaik1994/SalesForecasting_ML_CaseStudy/blob/main/notebooks/modelling/02.SARIMAX.ipynb?flush_cache=true)

In [233]:
import pandas as pd
import numpy as np
import duckdb as db
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import optuna
import warnings

warnings.filterwarnings('ignore')

from IPython.display import display, Markdown
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, MSTL, MFLES, AutoMFLES
# ConformalIntervals
from statsforecast.models import ConformalIntervals
# mape
from sklearn.metrics import mean_absolute_percentage_error
# sarimax
from statsmodels.tsa.statespace.sarimax import SARIMAX

# pd.set_option('plotting.backend', 'plotly')
# pio.renderers.default = "notebook_connected"
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [209]:
orig = pd.read_parquet("../../data/processed/train_enhanced.parquet")
train_agg = pd.read_parquet("../../data/processed/train_agg.parquet")
train_region_code_agg = pd.read_parquet("../../data/processed/train_region_code_agg.parquet")
holiday_df= pd.read_csv("../../data/processed/holidays.csv")

In [210]:
train_region_code_agg

,Date,Region_Code,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
0,2018-01-01,R4,2286812,45736,2914,58,50,1,50
1,2018-01-01,R2,4436859,42256,5644,54,105,1,105
2,2018-01-01,R3,3527439,41017,4599,53,86,1,86
3,2018-01-01,R1,5094374,41084,6509,52,124,1,124
4,2018-01-02,R4,2545119,50902,3057,61,50,0,50
...,...,...,...,...,...,...,...,...,...
2059,2019-05-30,R4,1966320,39326,2829,57,50,0,4
2060,2019-05-31,R2,4351299,41441,6411,61,105,1,11
2061,2019-05-31,R4,1909319,38186,2746,55,50,1,1
2062,2019-05-31,R1,5900798,47587,9433,76,124,1,18


In [211]:
df = train_region_code_agg[["Region_Code" ,"Date", "Total_Sales" , "Holiday","Total_Discounts"]]

In [212]:
date_mapping = {date: idx + 1 for idx, date in enumerate(sorted(df['Date'].unique()))}
df['idx'] = df['Date'].map(date_mapping)
df = df.rename(columns={"Total_Sales": "y", "Region_Code": "unique_id", "Date": "ds"})
df=df.sort_values(by='ds')
df

,unique_id,ds,y,Holiday,Total_Discounts,idx
0,R4,2018-01-01,2286812,1,50,1
1,R2,2018-01-01,4436859,1,105,1
2,R3,2018-01-01,3527439,1,86,1
3,R1,2018-01-01,5094374,1,124,1
4,R4,2018-01-02,2545119,0,50,2
...,...,...,...,...,...,...
2059,R4,2019-05-30,1966320,0,4,515
2061,R4,2019-05-31,1909319,1,1,516
2062,R1,2019-05-31,5900798,1,18,516
2060,R2,2019-05-31,4351299,1,11,516


In [213]:
threshold = 0.8
train = df[df['idx'] <= df['idx'].max() * threshold]
test = df[df['idx'] > df['idx'].max() * threshold]

In [214]:
train

,unique_id,ds,y,Holiday,Total_Discounts,idx
0,R4,2018-01-01,2286812,1,50,1
1,R2,2018-01-01,4436859,1,105,1
2,R3,2018-01-01,3527439,1,86,1
3,R1,2018-01-01,5094374,1,124,1
4,R4,2018-01-02,2545119,0,50,2
...,...,...,...,...,...,...
1640,R3,2019-02-15,3300873,0,73,411
1644,R2,2019-02-16,4485144,0,93,412
1645,R4,2019-02-16,2120472,0,49,412
1646,R1,2019-02-16,6426930,0,108,412


In [215]:
test

,unique_id,ds,y,Holiday,Total_Discounts,idx
1648,R3,2019-02-17,4253736,0,83,413
1649,R1,2019-02-17,6858420,0,120,413
1650,R4,2019-02-17,2341383,0,49,413
1651,R2,2019-02-17,4888986,0,99,413
1652,R3,2019-02-18,3948027,0,78,414
...,...,...,...,...,...,...
2059,R4,2019-05-30,1966320,0,4,515
2061,R4,2019-05-31,1909319,1,1,516
2062,R1,2019-05-31,5900798,1,18,516
2060,R2,2019-05-31,4351299,1,11,516


In [216]:
prediction_window = test['idx'].max() - train['idx'].max()
prediction_window

104

In [217]:
train = train.drop(columns=['idx'], axis=1)
test = test.drop(columns=['idx'], axis=1)

In [218]:
?MFLES

Init signature:
MFLES(
    season_length: Union[int, List[int], NoneType] = None,
    fourier_order: Optional[int] = None,
    max_rounds: int = 50,
    ma: Optional[int] = None,
    alpha: float = 1.0,
    decay: float = -1.0,
    changepoints: bool = True,
    n_changepoints: Union[float, int] = 0.25,
    seasonal_lr: float = 0.9,
    trend_lr: float = 0.9,
    exogenous_lr: float = 1.0,
    residuals_lr: float = 1.0,
    cov_threshold: float = 0.7,
    moving_medians: bool = False,
    min_alpha: float = 0.05,
    max_alpha: float = 1.0,
    trend_penalty: bool = True,
    multiplicative: Optional[bool] = None,
    smoother: bool = False,
    robust: Optional[bool] = None,
    verbose: bool = False,
    prediction_intervals: Optional[statsforecast.utils.ConformalIntervals] = None,
    alias: str = 'MFLES',
)
Docstring:     
MFLES model.

A method to forecast time series based on Gradient Boosted Time Series Decomposition
which treats traditional decomposition as the base estimator i

In [328]:
seasonality = [ 12, 30]
models = [
    AutoARIMA(
        season_length=12,
        d=0,
        approximation=True,
        stationary=True
    ),
    MSTL(season_length=[7, 12, 30],
         trend_forecaster=AutoARIMA(
        d=0,
        approximation=True,
        stationary=True,
        prediction_intervals=ConformalIntervals(h=prediction_window),
    )),

    AutoMFLES(
        season_length=seasonality,
        test_size=20,
        prediction_intervals=ConformalIntervals(h=prediction_window),
    )
]

In [329]:
fcst = StatsForecast( models=models, freq="D", n_jobs=-1)

In [330]:
fcst.fit(train);

In [331]:
forecast = fcst.predict(h=prediction_window, level=[95], X_df=test.drop(columns=['y'], axis=1))

In [332]:
forecast

,unique_id,ds,AutoARIMA,AutoARIMA-lo-95,AutoARIMA-hi-95,MSTL,MSTL-lo-95,MSTL-hi-95,AutoMFLES,AutoMFLES-lo-95,AutoMFLES-hi-95
0,R1,2019-02-17,6665677.000,4.966780e+06,8364574.00,7860963.500,7.173004e+06,8548923.000,6389826.500,5671097.000,7108556.000
1,R1,2019-02-18,6439911.000,4.581093e+06,8298729.50,6418570.500,5.809900e+06,7027240.500,6149990.000,5768254.500,6531725.500
2,R1,2019-02-19,4710731.500,2.821986e+06,6599477.00,3911237.750,1.795879e+06,6026596.500,4428278.000,1210137.625,7646418.000
3,R1,2019-02-20,5545148.000,3.650558e+06,7439737.50,5014326.500,3.016612e+06,7012041.000,5203267.500,2091442.250,8315093.000
4,R1,2019-02-21,5377810.000,3.482070e+06,7273549.00,4978095.000,2.719740e+06,7236449.000,5077038.000,1780144.125,8373931.500
...,...,...,...,...,...,...,...,...,...,...,...
411,R4,2019-05-27,2243409.000,1.553332e+06,2933486.50,2056336.500,1.985074e+06,2127598.750,1977072.875,1787338.250,2166807.500
412,R4,2019-05-28,2233841.500,1.543764e+06,2923919.00,1933951.875,1.730610e+06,2137294.000,1967190.875,1790080.125,2144301.750
413,R4,2019-05-29,2052060.500,1.361983e+06,2742138.00,1975758.500,1.719196e+06,2232321.000,1779434.500,1662350.375,1896518.625
414,R4,2019-05-30,1860712.125,1.170635e+06,2550789.75,1572959.375,1.179903e+06,1966015.875,1581796.250,1254850.625,1908741.875


In [333]:
merged =test.merge(forecast, on=['unique_id', 'ds'], how='left')
print("MAPE using ARIMA: ", mean_absolute_percentage_error(merged['y'], merged['AutoARIMA']))
print("MAPE using MSTL: ", mean_absolute_percentage_error(merged['y'], merged['MSTL']))
print("MAPE using AutoMFLES: ", mean_absolute_percentage_error(merged['y'], merged['AutoMFLES']))

MAPE using ARIMA:  0.17422901208870675
MAPE using MSTL:  0.17424597681344803
MAPE using AutoMFLES:  0.18700868651097424


In [267]:
def plot(train, test, forecast, unique_id, model_name):
    filtered_train = train[train['unique_id'] == unique_id]
    filtered_test = test[test['unique_id'] == unique_id]
    filtered_forecast = forecast[forecast['unique_id'] == unique_id]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=filtered_train['ds'], y=filtered_train['y'], mode='lines', name='Train'))
    fig.add_trace(go.Scatter(x=filtered_test['ds'], y=filtered_test['y'], mode='lines', name='Test'))
    fig.add_trace(go.Scatter(x=filtered_forecast['ds'], y=filtered_forecast[model_name], mode='lines', name='Forecast'))
    fig.add_trace(go.Scatter(
        x=filtered_forecast['ds'], 
        y=filtered_forecast[f"{model_name}-hi-95"], 
        mode='lines', 
        name='Upper Bound',
        line=dict(width=0),
        showlegend=False
    ))

    # Add lower bound as an area plot
    fig.add_trace(go.Scatter(
        x=filtered_forecast['ds'], 
        y=filtered_forecast[f"{model_name}-lo-95"], 
        mode='lines', 
        name='Lower Bound',
        fill='tonexty',  # Fill area between this trace and the previous one
        fillcolor='rgba(0, 100, 80, 0.2)',  # Set fill color with opacity
        line=dict(width=0),
        showlegend=False
    ))


    fig.update_layout(title=f'Total Sales Forecast for {unique_id}', xaxis_title='Date', yaxis_title='Total Sales')
    fig.show()

plot(train, test, forecast, "R1", "AutoARIMA")
plot(train, test, forecast, "R2", "AutoARIMA")
plot(train, test, forecast, "R3", "AutoARIMA")
plot(train, test, forecast, "R4", "AutoARIMA")

KeyError: 'AutoARIMA'

In [269]:
plot(train, test, forecast, "R1", "MSTL")
plot(train, test, forecast, "R2", "MSTL")
plot(train, test, forecast, "R3", "MSTL")
plot(train, test, forecast, "R4", "MSTL")

In [277]:
plot(train, test, forecast, "R1", "AutoMFLES")
plot(train, test, forecast, "R2", "AutoMFLES")
plot(train, test, forecast, "R3", "AutoMFLES")
plot(train, test, forecast, "R4", "AutoMFLES")